In [2]:
import pyodbc
import pandas as pd
import json

# ===========================================
# CONFIGURACIÓN DE CONEXIÓN
# ===========================================
server = "10.75.71.10,1433"
database = "UnoEE"

with open("config.json") as f:
    creds = json.load(f)

username = creds["DB_USER"]
password = creds["DB_PASS"]

# ===========================================
# CARGA CSV CON REFERENCIAS
# ===========================================
df_refs = pd.read_csv("referencias.csv", dtype=str)
refs_list = df_refs["Ref"].dropna().unique().tolist()

print(f"🔎 Se cargaron {len(refs_list)} referencias del CSV")

# ===========================================
# FUNCIÓN PARA ARMAR QUERY POR CHUNK (solo Y1 y Y3)
# ===========================================

def build_query(subset):
    values = ",".join([f"('{r}')" for r in subset])
    return f"""
    SET NOCOUNT ON;
    DECLARE @YearNow INT = YEAR(GETDATE());

    -- InputRefs con lista de referencias
    WITH
    InputRefs AS (
        SELECT Ref FROM (VALUES {values}) v(Ref)
    ),
    RefBase AS (
        -- Alternas distintas a la principal
        SELECT DISTINCT
            t124.f124_referencia AS Referencia_Alterna,
            t120.f120_referencia AS Referencia_Principal
        FROM t124_mc_items_referencias t124
        JOIN t120_mc_items t120 ON t124.f124_rowid_item = t120.f120_rowid
        WHERE t124.f124_referencia <> t120.f120_referencia

        UNION
        -- Principales sin alterna → alterna = principal
        SELECT 
            t120.f120_referencia AS Referencia_Alterna,
            t120.f120_referencia AS Referencia_Principal
        FROM t120_mc_items t120
        WHERE NOT EXISTS (
            SELECT 1
            FROM t124_mc_items_referencias r
            WHERE r.f124_rowid_item = t120.f120_rowid
              AND r.f124_referencia <> t120.f120_referencia
        )
    ),
    DetectRef AS (
        SELECT r.Ref,
               CASE 
                 WHEN EXISTS (SELECT 1 FROM RefBase WHERE Referencia_Principal = r.Ref) THEN 1 
                 ELSE 0 
               END AS FlagPrincipal
        FROM InputRefs r
    ),
    BaseRefs AS (
        SELECT 
            CASE 
                WHEN d.FlagPrincipal = 1 
                    THEN r.Ref                  -- si es principal: la misma
                ELSE rb.Referencia_Principal   -- si es alterna: buscamos su principal
            END AS Referencia_Principal,

            CASE 
                WHEN d.FlagPrincipal = 1 
                    THEN r.Ref   -- si es principal: se usa como alterna también
                ELSE r.Ref      -- si es alterna: se queda solo esa alterna
            END AS Referencia_Alterna
        FROM InputRefs r
        JOIN DetectRef d ON r.Ref = d.Ref
        LEFT JOIN RefBase rb 
               ON r.Ref = rb.Referencia_Alterna AND d.FlagPrincipal = 0
    ),
    -- =======================================
    -- Compras y agregaciones (igual que antes)
    -- =======================================
    Compras AS (
        SELECT 
            i.f120_referencia AS Referencia_Principal,
            YEAR(imp.f41850_fecha) AS Anho,
            CASE 
                WHEN p.f011_id IN ('249','840') THEN 'USA'
                WHEN p.f011_id IN ('076','105') THEN 'BR'
                ELSE 'OTROS'
            END AS Origen,
            SUM(mov.f41851_cant_entrada) AS Unidades,
            SUM(mov.f41851_cant_entrada * movoc.f421_precio_unitario) AS Valor_USD
        FROM t41851_import_movto          AS mov
        JOIN t41850_import_docto          AS imp   ON mov.f41851_rowid_docto_import = imp.f41850_rowid
        JOIN t41806_import_origen_destino AS od    ON imp.f41850_rowid_origen = od.f41806_rowid
        JOIN t011_mm_paises               AS p     ON od.f41806_id_pais = p.f011_id
        JOIN t121_mc_items_extensiones    AS e     ON mov.f41851_rowid_item_ext = e.f121_rowid
        JOIN t120_mc_items                AS i     ON e.f121_rowid_item = i.f120_rowid
        JOIN t421_cm_oc_movto             AS movoc ON mov.f41851_rowid_oc_movto = movoc.f421_rowid
        JOIN BaseRefs                     AS br    ON br.Referencia_Principal = i.f120_referencia
        WHERE imp.f41850_ind_estado    = 4
          AND imp.f41850_id_tipo_docto = 'IM'
        GROUP BY i.f120_referencia,
                 YEAR(imp.f41850_fecha),
                 CASE 
                    WHEN p.f011_id IN ('249','840') THEN 'USA'
                    WHEN p.f011_id IN ('076','105') THEN 'BR'
                    ELSE 'OTROS'
                 END
    ),
    AggPer AS (
        SELECT Referencia_Principal, 'Y1' AS Periodo, Origen,
               SUM(Unidades) AS U, SUM(Valor_USD) AS VUSD
        FROM Compras
        WHERE Anho = @YearNow
        GROUP BY Referencia_Principal, Origen
        UNION ALL
        SELECT Referencia_Principal, 'Y3', Origen,
               SUM(Unidades), SUM(Valor_USD)
        FROM Compras
        WHERE Anho BETWEEN @YearNow-2 AND @YearNow
        GROUP BY Referencia_Principal, Origen
    ),
    TotPer AS (
        SELECT Referencia_Principal, Periodo,
               SUM(U) AS U_TOT, SUM(VUSD) AS VUSD_TOT
        FROM AggPer
        GROUP BY Referencia_Principal, Periodo
    ),
    TotWide AS (
        SELECT
          t.Referencia_Principal,
          MAX(CASE WHEN t.Periodo='Y1' THEN t.U_TOT END) AS Unidades_Y1,
          MAX(CASE WHEN t.Periodo='Y3' THEN t.U_TOT END) AS Unidades_Y3,
          MAX(CASE WHEN t.Periodo='Y1' THEN t.VUSD_TOT END) AS ValorUSD_Y1,
          MAX(CASE WHEN t.Periodo='Y3' THEN t.VUSD_TOT END) AS ValorUSD_Y3
        FROM TotPer t
        GROUP BY t.Referencia_Principal
    ),
    Pct AS (
        SELECT a.Referencia_Principal, a.Periodo, a.Origen,
               ISNULL(CAST(100.0*a.U/NULLIF(t.U_TOT,0) AS DECIMAL(10,2)),0) AS PctUnid,
               ISNULL(CAST(100.0*a.VUSD/NULLIF(t.VUSD_TOT,0) AS DECIMAL(10,2)),0) AS PctValor
        FROM AggPer a
        JOIN TotPer t
          ON t.Referencia_Principal = a.Referencia_Principal
         AND t.Periodo = a.Periodo
    ),
    PctWide AS (
        SELECT p.Referencia_Principal,
          MAX(CASE WHEN Periodo='Y1' AND Origen='USA'   THEN PctUnid END) AS PctUnid_Y1_USA,
          MAX(CASE WHEN Periodo='Y1' AND Origen='BR'    THEN PctUnid END) AS PctUnid_Y1_BR,
          MAX(CASE WHEN Periodo='Y1' AND Origen='OTROS' THEN PctUnid END) AS PctUnid_Y1_OTROS,
          MAX(CASE WHEN Periodo='Y3' AND Origen='USA'   THEN PctUnid END) AS PctUnid_Y3_USA,
          MAX(CASE WHEN Periodo='Y3' AND Origen='BR'    THEN PctUnid END) AS PctUnid_Y3_BR,
          MAX(CASE WHEN Periodo='Y3' AND Origen='OTROS' THEN PctUnid END) AS PctUnid_Y3_OTROS,
          MAX(CASE WHEN Periodo='Y1' AND Origen='USA'   THEN PctValor END) AS PctValor_Y1_USA,
          MAX(CASE WHEN Periodo='Y1' AND Origen='BR'    THEN PctValor END) AS PctValor_Y1_BR,
          MAX(CASE WHEN Periodo='Y1' AND Origen='OTROS' THEN PctValor END) AS PctValor_Y1_OTROS,
          MAX(CASE WHEN Periodo='Y3' AND Origen='USA'   THEN PctValor END) AS PctValor_Y3_USA,
          MAX(CASE WHEN Periodo='Y3' AND Origen='BR'    THEN PctValor END) AS PctValor_Y3_BR,
          MAX(CASE WHEN Periodo='Y3' AND Origen='OTROS' THEN PctValor END) AS PctValor_Y3_OTROS
        FROM Pct p
        GROUP BY p.Referencia_Principal
    )
    SELECT
        br.Referencia_Principal,
        br.Referencia_Alterna,
        ISNULL(tw.Unidades_Y1,0) AS Unidades_Y1,
        ISNULL(tw.Unidades_Y3,0) AS Unidades_Y3,
        ISNULL(tw.ValorUSD_Y1,0) AS ValorUSD_Y1,
        ISNULL(tw.ValorUSD_Y3,0) AS ValorUSD_Y3,
        ISNULL(pw.PctUnid_Y1_USA,0) AS PctUnid_Y1_USA,
        ISNULL(pw.PctUnid_Y1_BR,0) AS PctUnid_Y1_BR,
        ISNULL(pw.PctUnid_Y1_OTROS,0) AS PctUnid_Y1_OTROS,
        ISNULL(pw.PctUnid_Y3_USA,0) AS PctUnid_Y3_USA,
        ISNULL(pw.PctUnid_Y3_BR,0) AS PctUnid_Y3_BR,
        ISNULL(pw.PctUnid_Y3_OTROS,0) AS PctUnid_Y3_OTROS,
        ISNULL(pw.PctValor_Y1_USA,0) AS PctValor_Y1_USA,
        ISNULL(pw.PctValor_Y1_BR,0) AS PctValor_Y1_BR,
        ISNULL(pw.PctValor_Y1_OTROS,0) AS PctValor_Y1_OTROS,
        ISNULL(pw.PctValor_Y3_USA,0) AS PctValor_Y3_USA,
        ISNULL(pw.PctValor_Y3_BR,0) AS PctValor_Y3_BR,
        ISNULL(pw.PctValor_Y3_OTROS,0) AS PctValor_Y3_OTROS,
        CAST(CASE WHEN ISNULL(tw.Unidades_Y1,0) > 0 THEN 1 ELSE 0 END AS BIT) AS TieneCompras_Y1,
        CAST(CASE WHEN ISNULL(tw.Unidades_Y3,0) > 0 THEN 1 ELSE 0 END AS BIT) AS TieneCompras_Y3
    FROM BaseRefs br
    LEFT JOIN TotWide tw ON tw.Referencia_Principal = br.Referencia_Principal
    LEFT JOIN PctWide pw ON pw.Referencia_Principal = br.Referencia_Principal
    ORDER BY br.Referencia_Principal;
    """
# ===========================================
# CONEXIÓN Y EJECUCIÓN POR CHUNKS
# ===========================================
try:
    connection = pyodbc.connect(
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER={server};"
        f"DATABASE={database};"
        f"UID={username};"
        f"PWD={password};"
        f"Encrypt=yes;"
        f"TrustServerCertificate=yes;"
    )
    print("✅ Conexión exitosa")

    chunk_size = 1000
    dfs = []

    for i in range(0, len(refs_list), chunk_size):
        subset = refs_list[i:i+chunk_size]
        print(f"▶ Procesando refs {i+1} a {i+len(subset)}...")
        sql_query = build_query(subset)

        # Ejecutar con cursor manual (sin pd.read_sql)
        cursor = connection.cursor()
        cursor.execute(sql_query)
        rows = cursor.fetchall()
        cols = [col[0] for col in cursor.description]
        df_chunk = pd.DataFrame.from_records(rows, columns=cols)

        dfs.append(df_chunk)

    df_result = pd.concat(dfs, ignore_index=True)

    output_file = "resultado_referencias_Y1_Y3.xlsx"
    df_result.to_excel(output_file, index=False)
    print(f"📂 Resultado final guardado en: {output_file}")
    print(df_result.head())  # 👉 Opcional: ver primeras filas en consola

except Exception as e:
    print("❌ Error:", e)

finally:
    if 'connection' in locals():
        connection.close()


🔎 Se cargaron 9774 referencias del CSV
✅ Conexión exitosa
▶ Procesando refs 1 a 1000...
▶ Procesando refs 1001 a 2000...
▶ Procesando refs 2001 a 3000...
▶ Procesando refs 3001 a 4000...
▶ Procesando refs 4001 a 5000...
▶ Procesando refs 5001 a 6000...
▶ Procesando refs 6001 a 7000...
▶ Procesando refs 7001 a 8000...
▶ Procesando refs 8001 a 9000...
▶ Procesando refs 9001 a 9774...
📂 Resultado final guardado en: resultado_referencias_Y1_Y3.xlsx
  Referencia_Principal Referencia_Alterna Unidades_Y1 Unidades_Y3 ValorUSD_Y1  \
0                 None           00140983      0.0000      0.0000    0.000000   
1                 None           00606627      0.0000      0.0000    0.000000   
2                 None         0086698700      0.0000      0.0000    0.000000   
3                 None         0087254303      0.0000      0.0000    0.000000   
4                 None         0088109734      0.0000      0.0000    0.000000   

  ValorUSD_Y3 PctUnid_Y1_USA PctUnid_Y1_BR PctUnid_Y1_OTROS PctU